# BBMs for the Power Flow

In this script, we aim to construct Black-Box Models (BBMs) (i.e., data-driven models) for the Power Flow (PF) problem. We use the data stored in the 'data/PF' folder, containing tables with the PF IO data.

In [1]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import sys
from importlib import reload

import greyboxmodels.bbmcpsmodels.nn_creator as creator


# Set the working directory
os.chdir("D:/projects/IPTLC_BBMs/")
print(f"Working directory: {os.getcwd()}")

# Check GPU availability
device = creator.get_device()
print(f"Device: {device}")

Working directory: D:\projects\IPTLC_BBMs
Device: cuda:0


In [2]:
# Specify the paths
datasets_folder = Path("data/IO-datasets/PF/2024-03-20_18-55-20/")

# # Print the available datasets
# print("Available datasets:")
# for path in datasets_folder.iterdir():
#     if path.is_dir() and "OPF" in path.name:
#         print(path.name)

# Setup the BBM creator

In [3]:
# Create the BBMCreator
reload(creator)
BBM_creator = creator.BBMCreator()

In [7]:
# Set up the dataset
dataset_name = "MinMaxNormalizedPF"
loaders = creator.setup_datasets(datasets_folder,
                                 dataset_name,
                                 remove_nans=True,
                                 ratios=(0.70, 0.15, 0.15),
                                 batch_size=32,
                                 input_name="pf_inputs_minmax_normalized.npy",
                                 output_name="pf_outputs_minmax_normalized.npy")
BBM_creator.set_dataloaders(*loaders)

Loading: data\IO-datasets\PF\2024-03-20_18-55-20
---- Dataset loaded ----
    Input shape: (26660, 57)
    Output shape: (26660, 105)
---- Removing NaNs ----
    Rows to delete: []
    Input shape: (26660, 57)
    Output shape: (26660, 105)
---- Converting to torch tensors ----
---- Dataset loaded! ----


# BBM 1: a two-layer feedforward neural network

In [8]:
from greyboxmodels.bbmcpsmodels.physical.feedforward_nn_pf import BBM1_SimpleNet

In [9]:
# Instantiate model
input_size = loaders[0].dataset[0][0].shape[0]
output_size = loaders[0].dataset[0][1].shape[0]

BBM_creator.instantiate_model(BBM1_SimpleNet, input_size, output_size)

In [ ]:
# Train the model
BBM_creator.train(save_to="models", epochs=100)

------ Training model 'BBM1_SimpleNet' ------
Models and summary will be save to 'models/' (will be created if it does not exist)
    - Model path: models\BBM1_SimpleNet_MinMaxNormalizedPF_20240325-013033.pt
    - Summary path: models/models_summary.csv
Training on cuda:0


Epoch 1/100 (Loss - Train: 1.18e-02, Best val: N/A): 100%|██████████| 584/584 [00:06<00:00, 94.39it/s] 
Epoch 2/100 (Loss - Train: 3.26e-03, Best val: 4.66e-03): 100%|██████████| 584/584 [00:02<00:00, 290.92it/s]
Epoch 3/100 (Loss - Train: 2.45e-03, Best val: 2.84e-03): 100%|██████████| 584/584 [00:02<00:00, 273.35it/s]
Epoch 4/100 (Loss - Train: 2.10e-03, Best val: 2.24e-03): 100%|██████████| 584/584 [00:02<00:00, 285.25it/s]
Epoch 5/100 (Loss - Train: 1.90e-03, Best val: 2.11e-03): 100%|██████████| 584/584 [00:02<00:00, 260.98it/s]
Epoch 6/100 (Loss - Train: 1.69e-03, Best val: 1.79e-03): 100%|██████████| 584/584 [00:02<00:00, 260.93it/s]
Epoch 7/100 (Loss - Train: 1.64e-03, Best val: 1.79e-03): 100%|██████████| 584/584 [00:02<00:00, 252.82it/s]
Epoch 8/100 (Loss - Train: 1.50e-03, Best val: 1.54e-03): 100%|██████████| 584/584 [00:02<00:00, 246.28it/s]
Epoch 9/100 (Loss - Train: 1.46e-03, Best val: 1.47e-03): 100%|██████████| 584/584 [00:02<00:00, 249.02it/s]
Epoch 10/100 (Loss - Tra

In [ ]:
BBM_creator._summary()